<a href="https://colab.research.google.com/github/TrieuDinhHo/Brain-Tumor-Segmentation/blob/main/DataPreprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Unzip Data

In [ ]:
!unzip /content/drive/MyDrive/ANN/archive.zip

Archive:  /content/drive/MyDrive/ANN/archive.zip
  inflating: MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_10_1/BraTS19_2013_10_1_flair.nii  
  inflating: MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_10_1/BraTS19_2013_10_1_seg.nii  
  inflating: MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_10_1/BraTS19_2013_10_1_t1.nii  
  inflating: MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_10_1/BraTS19_2013_10_1_t1ce.nii  
  inflating: MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_10_1/BraTS19_2013_10_1_t2.nii  
  inflating: MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_11_1/BraTS19_2013_11_1_flair.nii  
  inflating: MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_11_1/BraTS19_2013_11_1_seg.nii  
  inflating: MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_11_1/BraTS19_2013_11_1_t1.nii  
  inflating: MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_11_1/BraTS19_2013_11_1_t1ce.nii  
  inflating: MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_11_1/BraTS19_2013_11_1_t2.nii  

# Visualize a MRI Image

In [5]:
import nibabel as nib
from ipywidgets import interact
from matplotlib import pyplot as plt
import numpy as np

# # Load the Nifti File
# image = nib.load('/content/BraTS2019_preprocessed/delete_me.nii.gz')

# # Convert the image data as a NumPy array
# array = image.get_fdata()
# print("Shape of data is",np.shape(array))
# # Visualize Images as a slider
# def show_slice(i):
#     plt.imshow(array[:,:,i])
#     plt.show()

# interact(show_slice, i=(0, array.shape[2]-1))

In [ ]:
# Download some important libraries
#!pip install --upgrade batchgenerators
!git clone https://github.com/FabianIsensee/BatchViewer.git
!git clone https://github.com/MIC-DKFZ/batchgenerators.git
!pip install -e /content/batchgenerators/
!pip install -e /content/BatchViewer/
!pip install SimpleITK

fatal: destination path 'BatchViewer' already exists and is not an empty directory.
fatal: destination path 'batchgenerators' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/batchgenerators
  Preparing metadata (setup.py) ... done
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
  Attempting uninstall: batchgenerators
    Found existing installation: batchgenerators 0.25
    Uninstalling batchgenerators-0.25:
      Successfully uninstalled batchgenerators-0.25
  Running setup.py develop for batchgenerators


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/BatchViewer
  Preparing metadata (setup.py) ... done
  Attempting uninstall: batchviewer
    Found existing installation: batchviewer 0.1
    Uninstalling batchviewer-0.1:
      Successfully uninstalled batchviewer-0.1
  Running setup.py develop for batchviewer
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 15.4 MB/s eta 0:00:00


# ***Config Directory Data***

In [7]:
brats_preprocessed_folder = "/content/BraTS2019_preprocessed"
brats_folder_with_downloaded_train_data = "/content/MICCAI_BraTS_2019_Data_Training"
num_threads_for_brats_example = 8

In [ ]:
import numpy as np
from batchgenerators.batchgenerators.utilities.file_and_folder_operations import *

try:
    import SimpleITK as sitk
except ImportError:
    print("You need to have SimpleITK installed to run this example!")
    raise ImportError("SimpleITK not found")

from multiprocessing import Pool


def get_list_of_files(base_dir):
    """
    returns a list of lists containing the filenames. The outer list contains all training examples. Each entry in the
    outer list is again a list pointing to the files of that training example in the following order:
    T1, T1c, T2, FLAIR, segmentation
    :param base_dir:
    :return:
    """
    list_of_lists = []
    for glioma_type in ['HGG', 'LGG']:
        current_directory = join(base_dir, glioma_type)
        patients = subfolders(current_directory, join=False)
        #print(patients)
        for p in patients:
            patient_directory = join(current_directory, p)
            t1_file = join(patient_directory, p + "_t1.nii")
            t1c_file = join(patient_directory, p + "_t1ce.nii")
            t2_file = join(patient_directory, p + "_t2.nii")
            flair_file = join(patient_directory, p + "_flair.nii")
            seg_file = join(patient_directory, p + "_seg.nii")
            this_case = [t1_file, t1c_file, t2_file, flair_file, seg_file]
            assert all((isfile(i) for i in this_case)), "some file is missing for patient %s; make sure the following " \
                                                        "files are there: %s" % (p, str(this_case))
            list_of_lists.append(this_case)
    print("Found %d patients" % len(list_of_lists))
    return list_of_lists


def load_and_preprocess(case, patient_name, output_folder):
    """
    loads, preprocesses and saves a case
    This is what happens here:
    1) load all images and stack them to a 4d array
    2) crop to nonzero region, this removes unnecessary zero-valued regions and reduces computation time
    3) normalize the nonzero region with its mean and standard deviation
    4) save 4d tensor as numpy array. Also save metadata required to create niftis again (required for export
    of predictions)

    :param case:
    :param patient_name:
    :return:
    """
    # load SimpleITK Images
    imgs_sitk = [sitk.ReadImage(i) for i in case]

    # get pixel arrays from SimpleITK images
    imgs_npy = [sitk.GetArrayFromImage(i) for i in imgs_sitk]

    # get some metadata
    spacing = imgs_sitk[0].GetSpacing()
    # the spacing returned by SimpleITK is in inverse order relative to the numpy array we receive. If we wanted to
    # resample the data and if the spacing was not isotropic (in BraTS all cases have already been resampled to 1x1x1mm
    # by the organizers) then we need to pay attention here. Therefore we bring the spacing into the correct order so
    # that spacing[0] actually corresponds to the spacing of the first axis of the numpy array
    spacing = np.array(spacing)[::-1]

    direction = imgs_sitk[0].GetDirection()
    origin = imgs_sitk[0].GetOrigin()

    original_shape = imgs_npy[0].shape

    # now stack the images into one 4d array, cast to float because we will get rounding problems if we don't
    imgs_npy = np.concatenate([i[None] for i in imgs_npy]).astype(np.float32)

    # now find the nonzero region and crop to that
    nonzero = [np.array(np.where(i != 0)) for i in imgs_npy]
    nonzero = [[np.min(i, 1), np.max(i, 1)] for i in nonzero]
    nonzero = np.array([np.min([i[0] for i in nonzero], 0), np.max([i[1] for i in nonzero], 0)]).T
    # nonzero now has shape 3, 2. It contains the (min, max) coordinate of nonzero voxels for each axis

    # now crop to nonzero
    imgs_npy = imgs_npy[:,
               nonzero[0, 0] : nonzero[0, 1] + 1,
               nonzero[1, 0]: nonzero[1, 1] + 1,
               nonzero[2, 0]: nonzero[2, 1] + 1,
               ]

    # now we create a brain mask that we use for normalization
    nonzero_masks = [i != 0 for i in imgs_npy[:-1]]
    brain_mask = np.zeros(imgs_npy.shape[1:], dtype=bool)
    for i in range(len(nonzero_masks)):
        brain_mask = brain_mask | nonzero_masks[i]

    # now normalize each modality with its mean and standard deviation (computed within the brain mask)
    for i in range(len(imgs_npy) - 1):
        mean = imgs_npy[i][brain_mask].mean()
        std = imgs_npy[i][brain_mask].std()
        imgs_npy[i] = (imgs_npy[i] - mean) / (std + 1e-8)
        imgs_npy[i][brain_mask == 0] = 0

    # the segmentation of brats has the values 0, 1, 2 and 4. This is pretty inconvenient to say the least.
    # We move everything that is 4 to 3
    imgs_npy[-1][imgs_npy[-1] == 4] = 3

    # now save as npz
    np.save(join(output_folder, patient_name + ".npy"), imgs_npy)

    metadata = {
        'spacing': spacing,
        'direction': direction,
        'origin': origin,
        'original_shape': original_shape,
        'nonzero_region': nonzero
    }

    save_pickle(metadata, join(output_folder, patient_name + ".pkl"))


def save_segmentation_as_nifti(segmentation, metadata, output_file):
    original_shape = metadata['original_shape']
    seg_original_shape = np.zeros(original_shape, dtype=np.uint8)
    nonzero = metadata['nonzero_region']
    seg_original_shape[nonzero[0, 0] : nonzero[0, 1] + 1,
               nonzero[1, 0]: nonzero[1, 1] + 1,
               nonzero[2, 0]: nonzero[2, 1] + 1] = segmentation
    sitk_image = sitk.GetImageFromArray(seg_original_shape)
    sitk_image.SetDirection(metadata['direction'])
    sitk_image.SetOrigin(metadata['origin'])
    # remember to revert spacing back to sitk order again
    sitk_image.SetSpacing(tuple(metadata['spacing'][[2, 1, 0]]))
    sitk.WriteImage(sitk_image, output_file)


if __name__ == "__main__":
    # This is the same preprocessing I used for our contributions to the BraTS 2017 and 2018 challenges.
    # Preprocessing is described in the documentation of load_and_preprocess

    # The training data is identical between BraTS 2017 and 2018. You can request access here:
    # https://ipp.cbica.upenn.edu/#BraTS18_registration

    # brats_base points to where the extracted downloaded training data is

    # preprocessed data is saved as npy. This may seem odd if you are familiar with medical images, but trust me it's
    # the best way to do this for deep learning. It does not make much of a difference for BraTS, but if you are
    # dealing with larger images this is crusial for your pipelines to not get stuck in CPU bottleneck. What we can do
    # with numpy arrays is we can load them via np.load(file, mmap_mode="r") and then read just parts of it on the fly
    # during training. This is super important if your patch size is smaller than the size of the entire patient (for
    # example if you work with large CT data or if you need 2D slices).
    # For this to work properly the output_folder (or wherever the data is stored during training) must be on an SSD!
    # HDDs are usually too slow and you also wouldn't want to do this over a network share (there are exceptions but
    # take this as a rule of thumb)

    # Why is this not an IPython Notebook you may ask? Because I HATE IPython Notebooks. Simple :-)

    list_of_lists = get_list_of_files(brats_folder_with_downloaded_train_data)

    maybe_mkdir_p(brats_preprocessed_folder)

    patient_names = [i[0].split("/")[-2] for i in list_of_lists]

    p = Pool(processes=num_threads_for_brats_example)
    p.starmap(load_and_preprocess, zip(list_of_lists, patient_names, [brats_preprocessed_folder] * len(list_of_lists)))
    p.close()
    p.join()



    
    # # remember that we cropped the data before preprocessing. If we predict the test cases, we want to run the same
    # # preprocessing for them. We need to then put the segmentation back into its original position (due to cropping).
    # # Here is how you can do that:

    # # lets use Brats17_2013_0_1 for this example
    # img = np.load(join(brats_preprocessed_folder, "BraTS19_2013_0_1.npy"))
    # metadata = load_pickle(join(brats_preprocessed_folder, "BraTS19_2013_0_1.pkl"))
    # # remember that we changed the segmentation labels from 0, 1, 2, 4 to 0, 1, 2, 3. We need to change that back to
    # # get the correct format
    # img[-1][img[-1] == 3] = 4
    # save_segmentation_as_nifti(img[-1], metadata, join(brats_preprocessed_folder, "delete_me.nii.gz"))

Found 335 patients


In [13]:
from time import time
from batchgenerators.augmentations.crop_and_pad_augmentations import crop
from batchgenerators.dataloading.multi_threaded_augmenter import MultiThreadedAugmenter
from batchgenerators.transforms.abstract_transforms import Compose
from batchgenerators.utilities.data_splitting import get_split_deterministic
from batchgenerators.utilities.file_and_folder_operations import *
import numpy as np
from batchgenerators.dataloading.data_loader import DataLoader
from batchgenerators.augmentations.utils import pad_nd_image
from batchgenerators.transforms.spatial_transforms import SpatialTransform_2, MirrorTransform
from batchgenerators.transforms.color_transforms import BrightnessMultiplicativeTransform, GammaTransform
from batchgenerators.transforms.noise_transforms import GaussianNoiseTransform, GaussianBlurTransform


def get_list_of_patients(preprocessed_data_folder):
    npy_files = subfiles(preprocessed_data_folder, suffix=".npy", join=True)
    # remove npy file extension
    patients = [i[:-4] for i in npy_files]
    return patients


class BraTS2017DataLoader3D(DataLoader):
    def __init__(self, data, batch_size, patch_size, num_threads_in_multithreaded, seed_for_shuffle=1234,
                 return_incomplete=False, shuffle=True, infinite=True):
        """
        data must be a list of patients as returned by get_list_of_patients (and split by get_split_deterministic)

        patch_size is the spatial size the retured batch will have

        """
        super().__init__(data, batch_size, num_threads_in_multithreaded, seed_for_shuffle, return_incomplete, shuffle,
                         infinite)
        self.patch_size = patch_size
        self.num_modalities = 4
        self.indices = list(range(len(data)))

    @staticmethod
    def load_patient(patient):
        data = np.load(patient + ".npy", mmap_mode="r")
        metadata = load_pickle(patient + ".pkl")
        return data, metadata

    def generate_train_batch(self):
        # DataLoader has its own methods for selecting what patients to use next, see its Documentation
        idx = self.get_indices()
        patients_for_batch = [self._data[i] for i in idx]

        # initialize empty array for data and seg
        data = np.zeros((self.batch_size, self.num_modalities, *self.patch_size), dtype=np.float32)
        seg = np.zeros((self.batch_size, 1, *self.patch_size), dtype=np.float32)

        metadata = []
        patient_names = []

        # iterate over patients_for_batch and include them in the batch
        for i, j in enumerate(patients_for_batch):
            patient_data, patient_metadata = self.load_patient(j)

            # this will only pad patient_data if its shape is smaller than self.patch_size
            patient_data = pad_nd_image(patient_data, self.patch_size)

            # now random crop to self.patch_size
            # crop expects the data to be (b, c, x, y, z) but patient_data is (c, x, y, z) so we need to add one
            # dummy dimension in order for it to work (@Todo, could be improved)
            patient_data, patient_seg = crop(patient_data[:-1][None], patient_data[-1:][None], self.patch_size, crop_type="random")

            data[i] = patient_data[0]
            seg[i] = patient_seg[0]

            metadata.append(patient_metadata)
            patient_names.append(j)

        return {'data': data, 'seg':seg, 'metadata':metadata, 'names':patient_names}


def get_train_transform(patch_size):
    # we now create a list of transforms. These are not necessarily the best transforms to use for BraTS, this is just
    # to showcase some things
    tr_transforms = []

    # the first thing we want to run is the SpatialTransform. It reduces the size of our data to patch_size and thus
    # also reduces the computational cost of all subsequent operations. All subsequent operations do not modify the
    # shape and do not transform spatially, so no border artifacts will be introduced
    # Here we use the new SpatialTransform_2 which uses a new way of parameterizing elastic_deform
    # We use all spatial transformations with a probability of 0.2 per sample. This means that 1 - (1 - 0.1) ** 3 = 27%
    # of samples will be augmented, the rest will just be cropped
    tr_transforms.append(
        SpatialTransform_2(
            patch_size, [i // 2 for i in patch_size],
            do_elastic_deform=True, deformation_scale=(0, 0.25),
            do_rotation=True,
            angle_x=(- 15 / 360. * 2 * np.pi, 15 / 360. * 2 * np.pi),
            angle_y=(- 15 / 360. * 2 * np.pi, 15 / 360. * 2 * np.pi),
            angle_z=(- 15 / 360. * 2 * np.pi, 15 / 360. * 2 * np.pi),
            do_scale=True, scale=(0.75, 1.25),
            border_mode_data='constant', border_cval_data=0,
            border_mode_seg='constant', border_cval_seg=0,
            order_seg=1, order_data=3,
            random_crop=True,
            p_el_per_sample=0.1, p_rot_per_sample=0.1, p_scale_per_sample=0.1
        )
    )

    # now we mirror along all axes
    tr_transforms.append(MirrorTransform(axes=(0, 1, 2)))

    # brightness transform for 15% of samples
    tr_transforms.append(BrightnessMultiplicativeTransform((0.7, 1.5), per_channel=True, p_per_sample=0.15))

    # gamma transform. This is a nonlinear transformation of intensity values
    # (https://en.wikipedia.org/wiki/Gamma_correction)
    # tr_transforms.append(GammaTransform(gamma_range=(0.5, 2), invert_image=False, per_channel=True, p_per_sample=0.15))
    # we can also invert the image, apply the transform and then invert back
    # tr_transforms.append(GammaTransform(gamma_range=(0.5, 2), invert_image=True, per_channel=True, p_per_sample=0.15))

    # Gaussian Noise
    tr_transforms.append(GaussianNoiseTransform(noise_variance=(0, 0.05), p_per_sample=0.15))

    # blurring. Some BraTS cases have very blurry modalities. This can simulate more patients with this problem and
    # thus make the model more robust to it
    tr_transforms.append(GaussianBlurTransform(blur_sigma=(0.5, 1.5), different_sigma_per_channel=True,
                                               p_per_channel=0.5, p_per_sample=0.15))

    # now we compose these transforms together
    tr_transforms = Compose(tr_transforms)
    return tr_transforms


# if __name__ == "__main__"
# test thu load data theo batch hoat dong tot chua
patients = get_list_of_patients(brats_preprocessed_folder)

train, val = get_split_deterministic(patients, fold=0, num_splits=5, random_state=12345)

patch_size = (128, 128, 128)
batch_size = 2

# I recommend you don't use 'iteration oder all training data' as epoch because in patch based training this is
# really not super well defined. If you leave all arguments as default then each batch sill contain randomly
# selected patients. Since we don't care about epochs here we can set num_threads_in_multithreaded to anything.
dataloader = BraTS2017DataLoader3D(train, batch_size, patch_size, 1)

batch = next(dataloader)
# Check what batch return 
# print("length cua batch['data'] is",len(batch['data']))         # 2
# print("type cua batch['data'][0] is",type(batch['data'][0]))    # Numpy array
# print("shape cua batch['data'][0]", batch['data'][0].shape)     # (4,128,128,128)
# print("shape cua batch['data'][1]", batch['data'][1].shape)     # (4,128,128,128)
# print("lenght of batch['seg'] is",len(batch['seg']))            # 2
# print("Shape of segment_data[0] is",np.shape(batch['seg'][0]))  # (1,128,128,128)
# print("Shape of segment_data[1] is",np.shape(batch['seg'][1]))  # (1,128,128,128)

# VISUALIZE DATA AFTER PREPROCESSING
# try:
#   from BatchViewer.batchviewer import view_batch
#   #batch viewer can show up to 4d tensors. We can show only one sample, but that should be sufficient here
#   view_batch(batch['data'][0])
# except ImportError:
#     view_batch = None
#     print("you can visualize batches with batchviewer. It's a nice and handy tool. You can get it here: "
#           "https://github.com/FabianIsensee/BatchViewer")



# ket hop load data theo batch va argument
# now we have some DataLoader. Let's go an get some augmentations

# first let's collect all shapes, you will see why later
shapes = [BraTS2017DataLoader3D.load_patient(i)[0].shape[1:] for i in patients]
# print("shape data loader",shapes)
max_shape = np.max(shapes, 0)
max_shape = np.max((max_shape, patch_size), 0)

# we create a new instance of DataLoader. This one will return batches of shape max_shape. Cropping/padding is
# now done by SpatialTransform. If we do it this way we avoid border artifacts (the entire brain of all cases will
# be in the batch and SpatialTransform will use zeros which is exactly what we have outside the brain)
# this is viable here but not viable if you work with different data. If you work for example with CT scans that
# can be up to 500x500x500 voxels large then you should do this differently. There, instead of using max_shape you
# should estimate what shape you need to extract so that subsequent SpatialTransform does not introduce border
# artifacts
dataloader_train = BraTS2017DataLoader3D(train, batch_size, max_shape, num_threads_for_brats_example)

# during training I like to run a validation from time to time to see where I am standing. This is not a correct
# validation because just like training this is patch-based but it's good enough. We don't do augmentation for the
# validation, so patch_size is used as shape target here
dataloader_validation = BraTS2017DataLoader3D(val, batch_size, patch_size, max(1, num_threads_for_brats_example // 2))

tr_transforms = get_train_transform(patch_size)

# finally we can create multithreaded transforms that we can actually use for training
# we don't pin memory here because this is pytorch specific.
tr_gen = MultiThreadedAugmenter(dataloader_train, tr_transforms, num_processes=num_threads_for_brats_example,
                                num_cached_per_queue=3,
                                seeds=None, pin_memory=False)
# we need less processes for vlaidation because we dont apply transformations
val_gen = MultiThreadedAugmenter(dataloader_validation, None,
                                  num_processes=max(1, num_threads_for_brats_example // 2), num_cached_per_queue=1,
                                  seeds=None,
                                  pin_memory=False)

# lets start the MultiThreadedAugmenter. This is not necessary but allows them to start generating training
# batches while other things run in the main thread
tr_gen.restart()
val_gen.restart()

batch = next(tr_gen)
# # xem thu trong batch co gi
# for key, value in batch.items() :
#     print(key)                                                # data,seg,metadata,names
# print(batch['names'])
print("length cua batch['data'] is",len(batch['data']))         # 2
print("type cua batch['data'][0] is",type(batch['data'][0]))    # Numpy array
print("shape cua batch['data'][0]", batch['data'][0].shape)     # (4,128,128,128)
print("shape cua batch['data'][1]", batch['data'][1].shape)     # (4,128,128,128)
print("lenght of batch['seg'] is",len(batch['seg']))            # 2
print("Shape of segment_data[0] is",np.shape(batch['seg'][0]))  # (1,128,128,128)
print("Shape of segment_data[1] is",np.shape(batch['seg'][1]))  # (1,128,128,128)


# now if this was a network training you would run epochs like this (remember tr_gen and val_gen generate
# inifinite examples! Don't do "for batch in tr_gen:"!!!):
# num_batches_per_epoch = 10
# num_validation_batches_per_epoch = 3
# num_epochs = 5
# # let's run this to get a time on how long it takes
# time_per_epoch = []
# start = time()
# for epoch in range(num_epochs):
#     start_epoch = time()
#     for b in range(num_batches_per_epoch):
#         batch = next(tr_gen)
#         # do network training here with this batch

#     for b in range(num_validation_batches_per_epoch):
#         batch = next(val_gen)
#         # run validation here
#     end_epoch = time()
#     time_per_epoch.append(end_epoch - start_epoch)
# end = time()
# total_time = end - start
# print("Running %d epochs took a total of %.2f seconds with time per epoch being %s" %
#       (num_epochs, total_time, str(time_per_epoch)))

# if you notice that you have CPU usage issues, reduce the probability with which the spatial transformations are
# applied in get_train_transform (down to 0.1 for example). SpatialTransform is the most expensive transform

# if you wish to visualize some augmented examples, install batchviewer and uncomment this
# if view_batch is not None:
#     for _ in range(4):
#         batch = next(tr_gen)
#         view_batch(batch['data'][0], batch['seg'][0])
# else:
#     print("Cannot visualize batches, install batchviewer first. It's a nice and handy tool. You can get it here: "
#           "https://github.com/FabianIsensee/BatchViewer")

length cua batch['data'] is 2
type cua batch['data'][0] is <class 'numpy.ndarray'>
shape cua batch['data'][0] (4, 128, 128, 128)
shape cua batch['data'][1] (4, 128, 128, 128)
lenght of batch['seg'] is 2
Shape of segment_data[0] is (1, 128, 128, 128)
Shape of segment_data[1] is (1, 128, 128, 128)
